correct tickers from CRSP

# 1) Setup

## 1a) Library functions

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
from datetime import datetime, timedelta
from copy import copy

print("done")

done


## 1b) Helper functions

In [3]:
def ListUnion(l1, l2):
    union = list(set(l1) | set(l2))
    return union

# 2) CRSP Permnos
- https://wrds-www.wharton.upenn.edu/data-dictionary/crsp_a_indexes/dsp500list/
- Log in to the SAS cloud and navigate to: /wrds/crsp/sasdata/a_indexes/dsp500list.sas7bdat
- convert permnos to IBES tickers here https://wrds-www.wharton.upenn.edu/pages/get-data/linking-suite-wrds/ibes-crsp-link/

In [4]:
#crsp sas
wrds_tickers = pd.read_sas("CorrectMembers/dsp500list.sas7bdat")
wrds_tickers

,PERMNO,start,ending
0,10006.0,1957-03-01,1984-07-18
1,10030.0,1957-03-01,1969-01-08
2,10049.0,1925-12-31,1932-10-01
3,10057.0,1957-03-01,1992-07-02
4,10078.0,1992-08-20,2010-01-28
...,...,...,...
2007,93159.0,2012-07-31,2016-03-29
2008,93246.0,2021-03-22,2021-12-31
2009,93422.0,2010-07-01,2015-06-30
2010,93429.0,2017-03-01,2021-12-31


In [6]:
relevant_tickers = wrds_tickers[(wrds_tickers["ending"] >= "2000-01-01")]
tickers = relevant_tickers["PERMNO"]
relevant_tickers

,PERMNO,start,ending
4,10078.0,1992-08-20,2010-01-28
6,10104.0,1989-08-03,2021-12-31
7,10107.0,1994-06-07,2021-12-31
8,10108.0,2002-07-22,2005-08-11
10,10137.0,2000-12-11,2011-02-25
...,...,...,...
2007,93159.0,2012-07-31,2016-03-29
2008,93246.0,2021-03-22,2021-12-31
2009,93422.0,2010-07-01,2015-06-30
2010,93429.0,2017-03-01,2021-12-31


In [7]:

with open("CorrectMembers/SPX_CRSPMembers.txt", 'w') as f:
    for item in tickers:
        item = int(item)
        f.write("%s\n" % item)
print("done")

done


## 2b) more comprehensive ticker names
- All from WRDS: https://wrds-www.wharton.upenn.edu/data-dictionary/crsp_a_indexes/dsp500list/
- Go to wrds > crsp > a_stock, a_index, a_cc
- Get dsp500list.sas7bdat, dsenames.sas7bdat, ccmxpf_linktable.sas7bdat
- or monthly equivalent
- Note: using the TICKERS - although non-unique, will get you more complete data from WRDS/Compustat etc

In [14]:
#Get permnos for sp500 index members
msp500list = pd.read_sas("CorrectMembers/msp500list.sas7bdat")

#Get company identifiers
msenames = pd.read_sas("CorrectMembers/msenames.sas7bdat")
# if nameendt is missing then set to today date
msenames['NAMEENDT']=msenames['NAMEENDT'].fillna(pd.to_datetime('today'))
# Merge with SP500 data
msp500 = pd.merge(msp500list, msenames, how = 'left', on = 'PERMNO')

#link to compustat
ccmxpf = pd.read_sas("CorrectMembers/ccmxpf_linktable.sas7bdat")
# if linkenddt is missing then set to today date
ccmxpf['linkenddt']=ccmxpf['linkenddt'].fillna(pd.to_datetime('today'))
#link to CCM
msp500_ccmxpf = pd.merge(msp500, ccmxpf, how='left', left_on=['PERMNO'], right_on = "lpermno")
msp500_ccmxpf

,PERMNO,start,ending,NAMEDT,NAMEENDT,SHRCD,EXCHCD,SICCD,NCUSIP,TICKER,...,CUSIP,gvkey,linkprim,liid,linktype,lpermno,lpermco,USEDFLAG,linkdt,linkenddt
0,10006.0,1957-03-01,1984-07-18,1925-12-31,1954-05-31,10.0,1.0,3740.0,NaN,NaN,...,b'00080010',b'001010',b'C',b'00X',b'LU',10006.0,22156.0,1.0,1950-05-01,1962-01-30 00:00:00.000000
1,10006.0,1957-03-01,1984-07-18,1925-12-31,1954-05-31,10.0,1.0,3740.0,NaN,NaN,...,b'00080010',b'001010',b'P',b'01',b'LU',10006.0,22156.0,1.0,1962-01-31,1984-06-28 00:00:00.000000
2,10006.0,1957-03-01,1984-07-18,1954-06-01,1962-07-01,10.0,1.0,3740.0,NaN,NaN,...,b'00080010',b'001010',b'C',b'00X',b'LU',10006.0,22156.0,1.0,1950-05-01,1962-01-30 00:00:00.000000
3,10006.0,1957-03-01,1984-07-18,1954-06-01,1962-07-01,10.0,1.0,3740.0,NaN,NaN,...,b'00080010',b'001010',b'P',b'01',b'LU',10006.0,22156.0,1.0,1962-01-31,1984-06-28 00:00:00.000000
4,10006.0,1957-03-01,1984-07-18,1962-07-02,1968-01-01,10.0,1.0,3743.0,NaN,b'ACF',...,b'00080010',b'001010',b'C',b'00X',b'LU',10006.0,22156.0,1.0,1950-05-01,1962-01-30 00:00:00.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15991,93429.0,2017-03-01,2021-12-31,2017-10-18,2018-09-16,11.0,3.0,9999.0,b'12503M10',b'CBOE',...,b'12503M10',b'184500',b'P',b'01',b'LC',93429.0,53447.0,1.0,2010-06-15,2022-02-14 11:04:07.582706
15992,93429.0,2017-03-01,2021-12-31,2018-09-17,2021-03-28,11.0,5.0,6231.0,b'12503M10',b'CBOE',...,b'12503M10',b'184500',b'P',b'01',b'LC',93429.0,53447.0,1.0,2010-06-15,2022-02-14 11:04:07.582706
15993,93429.0,2017-03-01,2021-12-31,2021-03-29,2021-12-31,11.0,5.0,6211.0,b'12503M10',b'CBOE',...,b'12503M10',b'184500',b'P',b'01',b'LC',93429.0,53447.0,1.0,2010-06-15,2022-02-14 11:04:07.582706
15994,93436.0,2020-12-21,2021-12-31,2010-06-29,2017-02-01,11.0,3.0,9999.0,b'88160R10',b'TSLA',...,b'88160R10',b'184996',b'P',b'01',b'LC',93436.0,53453.0,1.0,2010-06-29,2022-02-14 11:04:07.582706


In [15]:
df_toby = msp500_ccmxpf[['PERMNO', 'start', 'ending', 'NAMEDT', 'NAMEENDT','TICKER', 'COMNAM','CUSIP', 'gvkey']]
df_toby = df_toby[df_toby["ending"] >= "2000-01-01"]
#convert cusip, gvkey, liid to str
df_toby["COMNAM"] = df_toby["COMNAM"].astype(str).str[2:-1]
df_toby["CUSIP"] = df_toby["CUSIP"].astype(str).str[2:-1]
df_toby["gvkey"] = df_toby["gvkey"].astype(str).str[2:-1]
df_toby["TICKER"] = df_toby["TICKER"].astype(str).str[2:-1]
df_toby.to_pickle("link_table.pkl")
df_toby.to_csv("link_table.csv")
df_toby

,PERMNO,start,ending,NAMEDT,NAMEENDT,TICKER,COMNAM,CUSIP,gvkey
29,10078.0,1992-08-20,2010-01-28,1986-03-04,2004-06-09,SUNW,SUN MICROSYSTEMS INC,86681020,012136
30,10078.0,1992-08-20,2010-01-28,2004-06-10,2007-08-26,SUNW,SUN MICROSYSTEMS INC,86681020,012136
31,10078.0,1992-08-20,2010-01-28,2007-08-27,2007-11-11,JAVA,SUN MICROSYSTEMS INC,86681020,012136
32,10078.0,1992-08-20,2010-01-28,2007-11-12,2007-12-10,JAVA,SUN MICROSYSTEMS INC,86681020,012136
33,10078.0,1992-08-20,2010-01-28,2007-12-11,2010-01-26,JAVA,SUN MICROSYSTEMS INC,86681020,012136
...,...,...,...,...,...,...,...,...,...
15991,93429.0,2017-03-01,2021-12-31,2017-10-18,2018-09-16,CBOE,C B O E GLOBAL MARKETS INC,12503M10,184500
15992,93429.0,2017-03-01,2021-12-31,2018-09-17,2021-03-28,CBOE,C B O E GLOBAL MARKETS INC,12503M10,184500
15993,93429.0,2017-03-01,2021-12-31,2021-03-29,2021-12-31,CBOE,C B O E GLOBAL MARKETS INC,12503M10,184500
15994,93436.0,2020-12-21,2021-12-31,2010-06-29,2017-02-01,TSLA,TESLA MOTORS INC,88160R10,184996


In [10]:
#remove unnecessary columns - including ticker, since that is non-unique
df_sp500 = msp500_ccmxpf[['PERMNO', "TICKER", 'COMNAM', 'NCUSIP','gvkey', 'liid', 'start', 'ending']]
df_sp500 = df_sp500[df_sp500["ending"] >= "2000-01-01"]
df_sp500_tickers = df_sp500.drop_duplicates("TICKER")
df_sp500 = df_sp500.drop_duplicates("PERMNO")

#convert cusip, gvkey, liid to str
df_sp500["COMNAM"] = df_sp500["COMNAM"].astype(str).str[2:-1]
df_sp500["NCUSIP"] = df_sp500["NCUSIP"].astype(str).str[2:-1]
df_sp500["gvkey"] = df_sp500["gvkey"].astype(str).str[2:-1]
df_sp500["liid"] = df_sp500["liid"].astype(str).str[2:-1]
df_sp500["TICKER"] = df_sp500["TICKER"].astype(str).str[2:-1]
df_sp500["PERMNO"] = df_sp500["PERMNO"].astype(int)
df_sp500.rename(columns={'PERMNO':'permno', "NCUSIP":"CUSIP","liid":"iid"}, inplace=True)

df_sp500_tickers["TICKER"] = df_sp500_tickers["TICKER"].astype(str).str[2:-1]
df_sp500_tickers["PERMNO"] = df_sp500_tickers["PERMNO"].astype(int)
df_sp500_tickers["gvkey"] = df_sp500_tickers["gvkey"].astype(str).str[2:-1]
df_sp500_tickers.rename(columns={'PERMNO':'permno', "NCUSIP":"CUSIP","liid":"iid"}, inplace=True)


df_sp500


,permno,TICKER,COMNAM,CUSIP,gvkey,iid,start,ending
29,10078,SUNW,SUN MICROSYSTEMS INC,86681010,012136,01,1992-08-20,2010-01-28
42,10104,ORCL,ORACLE SYSTEMS CORP,68389X10,012142,01,1989-08-03,2021-12-31
46,10107,MSFT,MICROSOFT CORP,59491810,012141,01,1994-06-07,2021-12-31
48,10108,SNDT,SUNGARD DATA SYSTEMS INC,86736310,012144,01,2002-07-22,2005-08-11
68,10137,,AMERICAN WATER WORKS & ELEC INC,,001279,00X,2000-12-11,2011-02-25
...,...,...,...,...,...,...,...,...
15981,93159,ESV,E N S C O PLC NEW,G3157S10,002270,01,2012-07-31,2016-03-29
15984,93246,GNRC,GENERAC HOLDINGS INC,36873610,183736,01,2021-03-22,2021-12-31
15985,93422,QEP,Q E P RESOURCES INC,74733V10,154357,01,2010-07-01,2015-06-30
15990,93429,CBOE,C B O E HOLDINGS INC,12503M10,184500,01,2017-03-01,2021-12-31


In [11]:
df_sp500_tickers["permno"].duplicated().sum()

503

In [12]:
#save identifiers
save_list = ["permno","COMNAM", "CUSIP", "gvkey", "iid"]
for name in save_list:
    save_name = "CorrectMembers/SPX_CRSPMembers_"+ name +".txt"
    with open(save_name, 'w') as f:
        for item in df_sp500[name]:
            f.write("%s\n" % item)
print("done")

done


In [13]:
#tickers for analyst estimates
with open("CorrectMembers/SPX_CRSPMembers_TICKERS.txt", 'w') as f:
    for item in df_sp500_tickers["TICKER"]:
        f.write("%s\n" % item)
print("done")

done


# 3) Financial Ratios
- https://wrds-www.wharton.upenn.edu/pages/get-data/financial-ratios-suite-wrds/financial-ratios-with-ibes-subscription/financial-ratios-firm-level-ibes/
- Also use this to get conversion from PERMNO to TICKER
- create month label to merge with macro data


In [32]:
df_FundamentalRatios = pd.read_csv("CorrectMembers/FundamentalRatios.csv")
df_FundamentalRatios["month"] = pd.to_datetime(df_FundamentalRatios["public_date"]).dt.to_period('M')
df_FundamentalRatios.rename(columns={'cusip':'CUSIP'}, inplace=True)
df_FundamentalRatios = df_FundamentalRatios.drop(labels = ["permno","gvkey"],axis = 1)
df_FundamentalRatios = df_FundamentalRatios.merge(df_sp500_tickers[["permno","gvkey","TICKER"]],on = "TICKER", how = "inner")
df_FundamentalRatios.sort_values("month")

,adate,qdate,public_date,bm,evm,pe_exi,ps,pcf,dpr,npm,...,accrual,ptb,PEG_trailing,PEG_1yrforward,PEG_ltgforward,TICKER,CUSIP,month,permno,gvkey
11697,1998/12/31,1999/09/30,2000/01/31,0.266,23.916,36.936,11.366,80.683,0.000,0.102,...,0.028,8.229,NaN,0.221,1.404,NVLS,67000810,2000-01,12067,014623
98538,1998/12/31,1999/09/30,2000/01/31,0.965,9.300,6.171,0.624,6.477,1.563,0.016,...,0.040,0.715,0.065,-0.187,0.606,CSX,12640810,2000-01,62148,002574
129841,1999/03/31,1999/09/30,2000/01/31,0.164,12.591,15.270,7.454,20.693,0.000,0.151,...,0.170,7.460,0.098,0.424,0.593,MCHP,59501710,2000-01,78987,027965
51535,1998/12/31,1999/09/30,2000/01/31,0.074,26.381,32.261,7.216,35.376,0.343,0.226,...,-0.024,13.598,NaN,1.655,2.044,SGP,80660510,2000-01,25013,009459
51654,1998/12/31,1999/09/30,2000/01/31,0.393,15.086,10.965,2.104,3.232,0.027,0.199,...,0.041,2.220,9.462,1.056,0.960,CMA,20034010,2000-01,25081,003231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132842,2019/12/31,2020/09/30,2020/12/31,0.658,8.582,NaN,0.958,4.099,0.040,0.079,...,0.144,1.785,NaN,NaN,NaN,UHS,91390310,2020-12,79637,011032
15996,2019/12/31,2020/09/30,2020/12/31,0.208,20.909,82.756,3.764,21.788,0.826,0.046,...,0.078,5.824,NaN,NaN,NaN,XYL,98419M10,2020-12,13035,189491
48330,2019/12/31,2020/09/30,2020/12/31,0.793,10.918,NaN,1.959,6.514,0.563,0.138,...,0.032,1.278,NaN,NaN,NaN,ETR,29364G10,2020-12,24010,007366
163597,2020/09/30,2020/09/30,2020/12/31,0.298,14.586,35.118,4.610,16.400,0.000,0.131,...,0.088,4.854,NaN,NaN,NaN,FFIV,31561610,2020-12,86964,121077


In [33]:
# get tickers
tickers = df_FundamentalRatios["TICKER"].unique()
with open("CorrectMembers/SPX_CRSPMembers_Tickers.txt", 'w') as f:
    for item in tickers:
        f.write("%s\n" % item)
print("done")

done


In [34]:
cusips = df_FundamentalRatios["CUSIP"].unique()
with open("CorrectMembers/SPX_CRSPMembers_Cusips.txt", 'w') as f:
    for item in cusips:
        f.write("%s\n" % item)
print("done")

done


# 4) Price Data
- compustat: https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/security-monthly/
- Optionmetrics: https://wrds-www.wharton.upenn.edu/pages/get-data/optionmetrics/ivy-db-us/securities/security-prices/

In [35]:
df_PriceData = pd.read_csv("CorrectMembers/PriceData.csv")
df_PriceData["month"] = pd.to_datetime(df_PriceData["datadate"]).dt.to_period('M')
df_PriceData.rename(columns={'tic':'TICKER'}, inplace=True)
df_PriceData["cusip"] = df_PriceData["cusip"].str[:-1]
df_PriceData.rename(columns={'cusip':'CUSIP'}, inplace=True)
df_PriceData = df_PriceData.drop(labels = ["gvkey"],axis = 1)
df_PriceData = df_PriceData.merge(df_sp500_tickers[["permno","gvkey","TICKER"]],on = "TICKER", how = "inner")
df_PriceData.sort_values(["month","TICKER"])

,iid,datadate,TICKER,CUSIP,conm,ajexm,cshtrm,curcdm,prccm,trfm,trt1m,cshom,ggroup,gind,gsector,gsubind,month,permno,gvkey
164902,01,2000/01/31,A,00846U10,AGILENT TECHNOLOGIES INC,1.0,32107900.0,USD,66.1875,1.0000,-14.3897,4.520000e+08,3520.0,352030.0,35.0,35203010.0,2000-01,87432,126554
5409,01,2000/01/31,AAPL,03783310,APPLE INC,112.0,111989500.0,USD,103.7500,1.0962,0.9119,1.611590e+08,4520.0,452020.0,45.0,45202030.0,2000-01,14593,001690
139070,01,2000/01/31,ABC,03073E10,AMERISOURCEBERGEN CORP,4.0,9007900.0,USD,18.1250,1.0000,19.3416,5.118800e+07,3510.0,351020.0,35.0,35102010.0,2000-01,81540,031673
98825,01,2000/01/31,ABMD,00365410,ABIOMED INC,2.0,1865200.0,USD,58.5000,1.0000,59.1837,8.708000e+06,3510.0,351010.0,35.0,35101010.0,2000-01,75107,013619
361,01,2000/01/31,ABT,00282410,ABBOTT LABORATORIES,1.0,124540200.0,USD,32.5625,2.2311,-9.8589,1.537311e+09,3510.0,351010.0,35.0,35101010.0,2000-01,20482,001078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171876,01,2021/12/31,ZBH,98956P10,ZIMMER BIOMET HOLDINGS INC,1.0,41701268.0,USD,127.0400,1.0875,6.4214,2.089080e+08,3510.0,351010.0,35.0,35101010.0,2021-12,89070,144559
116931,01,2021/12/31,ZBRA,98920710,ZEBRA TECHNOLOGIES CP -CL A,1.0,6187041.0,USD,595.2000,1.0000,1.0904,5.344100e+07,4520.0,452030.0,45.0,45203010.0,2021-12,76795,024405
88903,01,2021/12/31,ZION,98970110,ZIONS BANCORPORATION NA,1.0,27241303.0,USD,63.1600,3.5754,0.1268,1.564630e+08,4010.0,401010.0,40.0,40101015.0,2021-12,84129,011687
99976,01,2021/12/31,ZTS,98978V10,ZOETIS INC,1.0,30337184.0,USD,244.0300,1.0621,9.9036,4.731260e+08,3520.0,352020.0,35.0,35202010.0,2021-12,13788,013721


# 5) Analyst estimates
-  https://wrds-www.wharton.upenn.edu/pages/get-data/ibes-thomson-reuters/ibes-academic/unadjusted-summary/price-target/

In [36]:
df_AnalystEstimates = pd.read_csv("CorrectMembers/AnalystPredictions.csv")
df_AnalystEstimates["month"] = pd.to_datetime(df_AnalystEstimates["STATPERS"]).dt.to_period('M')
df_AnalystEstimates.rename(columns={'TICKER':'tic'}, inplace=True)
df_AnalystEstimates.rename(columns={'OFTIC':'TICKER'}, inplace=True)
df_AnalystEstimates = df_AnalystEstimates.merge(df_sp500_tickers[["permno","gvkey","TICKER"]],on = "TICKER", how = "inner")
df_AnalystEstimates.sort_values(["TICKER","month"])

,TICKER,tic,CUSIP,CNAME,STATPERS,NUMEST,NUMUP4W,NUMDOWN4W,MEDPTG,STDEV,PTGHIGH,PTGLOW,CURR,month,permno,gvkey
0,A,AT1,00846U10,AGILENT TECHNOLOGIES INC,2000/01/20,7,4,0,80.0,16.547,90.0,55.0,USD,2000-01,87432,126554
1,A,AT1,00846U10,AGILENT TECHNOLOGIES INC,2000/02/17,7,2,0,85.0,16.036,90.0,55.0,USD,2000-02,87432,126554
2,A,AT1,00846U10,AGILENT TECHNOLOGIES INC,2000/03/16,8,6,0,172.5,28.504,180.0,110.0,USD,2000-03,87432,126554
3,A,AT1,00846U10,AGILENT TECHNOLOGIES INC,2000/04/20,8,0,1,160.0,28.031,175.0,110.0,USD,2000-04,87432,126554
4,A,AT1,00846U10,AGILENT TECHNOLOGIES INC,2000/05/18,8,0,3,125.0,28.715,175.0,110.0,USD,2000-05,87432,126554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210358,ZY,03GA,98985X10,ZYMERGEN INC,2021/08/19,5,0,1,42.0,19.950,56.0,12.0,USD,2021-08,40539,011672
210359,ZY,03GA,98985X10,ZYMERGEN INC,2021/09/16,5,0,2,12.0,14.328,43.0,8.0,USD,2021-09,40539,011672
210360,ZY,03GA,98985X10,ZYMERGEN INC,2021/10/14,5,0,0,12.0,14.328,43.0,8.0,USD,2021-10,40539,011672
210361,ZY,03GA,98985X10,ZYMERGEN INC,2021/11/18,5,0,3,9.0,2.345,13.0,7.0,USD,2021-11,40539,011672


In [37]:
print(df_AnalystEstimates["tic"].nunique())
print(df_AnalystEstimates["TICKER"].nunique())
print(df_PriceData["TICKER"].nunique())
print(df_FundamentalRatios["TICKER"].nunique())

1303
1247
1066
1172


# 6) Merge dataframes + Macro data

In [226]:
len(ListUnion(df_FundamentalRatios["TICKER"].unique(),df_AnalystEstimates["TICKER"].unique()))

1277

In [255]:
#merge
df_Merge = copy(df_FundamentalRatios)
df_Merge = df_Merge.merge(df_AnalystEstimates, on = ["permno","month"], how = "inner")
df_Merge = df_Merge.merge(df_PriceData, on = ["permno","month"], how = "inner")
df_Merge.sort_values("month")

,adate,qdate,public_date,bm,evm,pe_exi,ps,pcf,dpr,npm,...,curcdm,prccm,trfm,trt1m,cshom,ggroup,gind,gsector,gsubind,gvkey
153354,1998/12/31,1999/09/30,2000/01/31,0.341,7.652,11.729,4.943,10.015,0.025,0.438,...,USD,46.5625,1.0322,-22.6376,105708000.0,4010.0,401020.0,40.0,40102010.0,024379
48406,1999/11/30,1999/11/30,2000/01/31,0.279,11.913,16.450,1.594,-3.212,0.040,0.107,...,USD,91.6250,1.0050,-2.5932,441271000.0,4020.0,402030.0,40.0,40203020.0,005169
91446,1998/12/31,1999/09/30,2000/01/31,0.883,7.792,12.089,0.763,5.000,0.045,0.065,...,USD,18.3750,10.1399,4.3077,125056000.0,5510.0,551030.0,55.0,55103010.0,007974
167590,1998/12/31,1999/09/30,2000/01/31,0.397,13.405,15.909,1.628,8.251,0.196,0.103,...,USD,42.0000,1.0962,2.4390,46710000.0,1510.0,151020.0,15.0,15102010.0,029733
86006,1998/12/31,1999/09/30,2000/01/31,0.535,21.132,NaN,3.472,25.181,0.000,-0.010,...,USD,3.9375,1.0614,173.9130,6925000.0,2010.0,201070.0,20.0,20107010.0,004367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164487,2019/12/31,2020/09/30,2020/12/31,0.238,18.049,32.579,9.368,35.350,0.143,0.289,...,USD,92.8500,1.1150,16.8218,772857000.0,5020.0,502020.0,50.0,50202020.0,001111
164804,2019/12/31,2020/09/30,2020/12/31,0.192,13.097,18.763,2.104,16.207,0.000,0.118,...,USD,4079.8600,1.4338,2.0685,3720000.0,2520.0,252010.0,25.0,25201030.0,012459
165055,2020/06/30,2020/09/30,2020/12/31,0.289,16.852,93.594,3.058,20.388,0.000,0.032,...,USD,14.9750,1.9032,10.5574,228994000.0,4520.0,452010.0,45.0,45201020.0,029241
165307,2020/01/31,2020/10/31,2020/12/31,0.655,21.330,-320.000,0.709,6.818,0.000,-0.002,...,USD,25.6000,1.0000,-6.5011,97786000.0,2550.0,255040.0,25.0,25504010.0,029150


In [256]:
#add in index inclusion
#df_Merge.insert(0, 'TICKER', df_Merge["TICKER_x"])
df_sp500.rename(columns={'PERMNO':'permno'}, inplace=True)
df_Merge = df_Merge.merge(df_sp500, on = ["permno"], how = "left")
df_Merge["is_member"] = ( (df_Merge["public_date"] >= df_Merge["start"]) & (df_Merge["public_date"] <= df_Merge["ending"]) )
df_Merge

C:\Users\pyliu\anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'CUSIP_x', 'gvkey_x', 'TICKER_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,adate,qdate,public_date,bm,evm,pe_exi,ps,pcf,dpr,npm,...,gsubind,gvkey_x,TICKER_y,COMNAM,CUSIP_y,gvkey_y,iid_y,start,ending,is_member
0,2008/12/31,2009/09/30,2009/11/30,0.850,5.945,11.708,0.498,3.113,0.000,0.042,...,55102010.0,004383,EGAS,ENERGAS CO,29293410,004383,01,2019-02-15,2021-12-31,False
1,2008/12/31,2009/09/30,2009/11/30,0.850,5.945,11.708,0.498,3.113,0.000,0.042,...,55102010.0,004383,EGAS,ENERGAS CO,29293410,004383,01,2019-02-15,2021-12-31,False
2,2009/09/30,2009/09/30,2009/11/30,1.054,7.298,13.168,0.510,2.759,0.636,0.038,...,55102010.0,004383,EGAS,ENERGAS CO,29293410,004383,01,2019-02-15,2021-12-31,False
3,2009/09/30,2009/09/30,2009/11/30,1.054,7.298,13.168,0.510,2.759,0.636,0.038,...,55102010.0,004383,EGAS,ENERGAS CO,29293410,004383,01,2019-02-15,2021-12-31,False
4,2008/12/31,2009/09/30,2009/12/31,0.850,5.945,13.553,0.577,3.604,0.000,0.042,...,55102010.0,004383,EGAS,ENERGAS CO,29293410,004383,01,2019-02-15,2021-12-31,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213397,2019/12/31,2020/06/30,2020/08/31,0.049,35.254,258.197,18.062,171.660,0.000,0.014,...,25102010.0,184996,TSLA,TESLA MOTORS INC,88160R10,184996,01,2020-12-21,2021-12-31,False
213398,2019/12/31,2020/06/30,2020/09/30,0.049,35.254,222.285,15.820,150.352,0.000,0.014,...,25102010.0,184996,TSLA,TESLA MOTORS INC,88160R10,184996,01,2020-12-21,2021-12-31,False
213399,2019/12/31,2020/06/30,2020/10/31,0.049,35.254,201.057,14.308,135.979,0.000,0.014,...,25102010.0,184996,TSLA,TESLA MOTORS INC,88160R10,184996,01,2020-12-21,2021-12-31,False
213400,2019/12/31,2020/09/30,2020/11/30,0.039,53.275,494.500,19.095,123.713,0.000,0.020,...,25102010.0,184996,TSLA,TESLA MOTORS INC,88160R10,184996,01,2020-12-21,2021-12-31,False


In [257]:
#(df_Merge.count()/len(df_Merge)).sort_values(ascending = False)[:20]
np.sort(df_Merge.columns)

array(['CNAME', 'COMNAM', 'CURR', 'CUSIP_x', 'CUSIP_x', 'CUSIP_y',
       'CUSIP_y', 'MEDPTG', 'NUMDOWN4W', 'NUMEST', 'NUMUP4W',
       'PEG_1yrforward', 'PEG_ltgforward', 'PEG_trailing', 'PTGHIGH',
       'PTGLOW', 'STATPERS', 'STDEV', 'TICKER_x', 'TICKER_x', 'TICKER_y',
       'TICKER_y', 'accrual', 'adate', 'aftret_invcapx', 'ajexm',
       'at_turn', 'bm', 'cash_conversion', 'conm', 'cshom', 'cshtrm',
       'curcdm', 'curr_ratio', 'datadate', 'de_ratio', 'debt_assets',
       'dpr', 'ending', 'evm', 'ggroup', 'gind', 'gsector', 'gsubind',
       'gvkey_x', 'gvkey_x', 'gvkey_y', 'gvkey_y', 'iid_x', 'iid_y',
       'is_member', 'month', 'npm', 'pcf', 'pe_exi', 'permno', 'prccm',
       'ps', 'ptb', 'public_date', 'qdate', 'quick_ratio', 'roe',
       'sale_nwc', 'start', 'tic', 'trfm', 'trt1m'], dtype=object)

In [258]:
temp = df_Merge.pop('month')
df_Merge.insert(0, 'month', temp)
df_Merge = df_Merge.loc[:,~df_Merge.columns.duplicated()]
df_Merge.insert(0, 'TICKER', df_Merge["TICKER_x"])
#df_Merge.insert(0, 'gvkey', df_Merge["gvkey_x"])
#df_Merge.insert(0, 'CUSIP', df_Merge["CUSIP_x"])
#df_Merge.insert(0, 'permno', df_Merge["permno_x"])
df_Merge = df_Merge.drop(labels = ["sale_nwc", "cash_conversion", "curr_ratio", "quick_ratio", "PEG_ltgforward", "dpr", "PEG_1yrforward", "PEG_trailing",
                        "qdate", "public_date", 'TICKER_x','TICKER_y',
                                   "CUSIP_x", "CUSIP_y", "datadate", "iid_x","iid_y","gvkey_x","gvkey_y","STATPERS","tic", "adate"],axis = 1)
df_Merge

,TICKER,month,bm,evm,pe_exi,ps,pcf,npm,roe,aftret_invcapx,...,trt1m,cshom,ggroup,gind,gsector,gsubind,COMNAM,start,ending,is_member
0,EGAS,2009-11,0.850,5.945,11.708,0.498,3.113,0.042,0.103,0.103,...,-0.4488,92600000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,2019-02-15,2021-12-31,False
1,EGAS,2009-11,0.850,5.945,11.708,0.498,3.113,0.042,0.103,0.103,...,0.7095,4361000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,2019-02-15,2021-12-31,False
2,ATO,2009-11,1.054,7.298,13.168,0.510,2.759,0.038,0.073,0.076,...,-0.4488,92600000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,2019-02-15,2021-12-31,False
3,ATO,2009-11,1.054,7.298,13.168,0.510,2.759,0.038,0.073,0.076,...,0.7095,4361000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,2019-02-15,2021-12-31,False
4,EGAS,2009-12,0.850,5.945,13.553,0.577,3.604,0.042,0.103,0.103,...,7.3384,92932000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,2019-02-15,2021-12-31,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213397,TSLA,2020-08,0.049,35.254,258.197,18.062,171.660,0.014,0.054,0.061,...,74.1452,931810000.0,2510.0,251020.0,25.0,25102010.0,TESLA MOTORS INC,2020-12-21,2021-12-31,False
213398,TSLA,2020-09,0.049,35.254,222.285,15.820,150.352,0.014,0.054,0.061,...,-13.9087,933540000.0,2510.0,251020.0,25.0,25102010.0,TESLA MOTORS INC,2020-12-21,2021-12-31,False
213399,TSLA,2020-10,0.049,35.254,201.057,14.308,135.979,0.014,0.054,0.061,...,-9.5499,947901000.0,2510.0,251020.0,25.0,25102010.0,TESLA MOTORS INC,2020-12-21,2021-12-31,False
213400,TSLA,2020-11,0.039,53.275,494.500,19.095,123.713,0.020,0.070,0.068,...,46.2736,947901000.0,2510.0,251020.0,25.0,25102010.0,TESLA MOTORS INC,2020-12-21,2021-12-31,False


In [259]:
macro = ["CPI", "FedFundsTargetRate", "GDP", "MedianHomeSalesPrice", 
         "NonFarmPayrolls", "PMI", "PPI", "PrivateHousingStarts", "Unemployment"]

for ratio in macro:
    readname = "MacroData/df_US_" + ratio + ".pkl"
    df_temp = pd.read_pickle(readname)
    if ratio == "PPI":   #PPI had different format
        df_temp = df_temp[1:]
        df_temp.columns = ["year", "m", "month", "PPI"]
        df_temp = df_temp[["month", "PPI"]]
        df_temp["month"] = pd.to_datetime(df_temp["month"]).dt.to_period('M')
    elif ratio == "GDP":
        df_temp.columns = ["date", ratio]
        df_temp["date"] = pd.to_datetime(df_temp["date"])   #format date
        df_temp["month"] = df_temp['date'].dt.to_period('M')   #nearest month
        df_temp = df_temp[["month", ratio]]
        df_temp = df_temp.set_index('month').resample('M').interpolate().reset_index()
    else:
        df_temp.columns = ["date", ratio]
        df_temp["date"] = pd.to_datetime(df_temp["date"])   #format date
        df_temp["month"] = df_temp['date'].dt.to_period('M')   #nearest month
        df_temp = df_temp[["month", ratio]]
    df_Merge = df_Merge.merge(df_temp, on = ["month"], how = "left")

df_Merge.to_pickle("df_Merge.pkl")
df_Merge

,TICKER,month,bm,evm,pe_exi,ps,pcf,npm,roe,aftret_invcapx,...,is_member,CPI,FedFundsTargetRate,GDP,MedianHomeSalesPrice,NonFarmPayrolls,PMI,PPI,PrivateHousingStarts,Unemployment
0,EGAS,2009-11,0.850,5.945,11.708,0.498,3.113,0.042,0.103,0.103,...,False,1.84,0.25,1.532499e+07,218800,130057000,54.4,177.4,0.64,9.9
1,EGAS,2009-11,0.850,5.945,11.708,0.498,3.113,0.042,0.103,0.103,...,False,1.84,0.25,1.532499e+07,218800,130057000,54.4,177.4,0.64,9.9
2,ATO,2009-11,1.054,7.298,13.168,0.510,2.759,0.038,0.073,0.076,...,False,1.84,0.25,1.532499e+07,218800,130057000,54.4,177.4,0.64,9.9
3,ATO,2009-11,1.054,7.298,13.168,0.510,2.759,0.038,0.073,0.076,...,False,1.84,0.25,1.532499e+07,218800,130057000,54.4,177.4,0.64,9.9
4,EGAS,2009-12,0.850,5.945,13.553,0.577,3.604,0.042,0.103,0.103,...,False,2.72,0.25,1.537916e+07,222600,129788000,55.3,178.1,0.63,9.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213397,TSLA,2020-08,0.049,35.254,258.197,18.062,171.660,0.014,0.054,0.061,...,False,1.31,0.25,1.812658e+07,325500,141149000,55.6,194.3,1.47,8.4
213398,TSLA,2020-09,0.049,35.254,222.285,15.820,150.352,0.014,0.054,0.061,...,False,1.37,0.25,1.856077e+07,344400,141865000,55.7,195.5,1.54,7.9
213399,TSLA,2020-10,0.049,35.254,201.057,14.308,135.979,0.014,0.054,0.061,...,False,1.18,0.25,1.862978e+07,346900,142545000,58.8,196.5,1.61,6.9
213400,TSLA,2020-11,0.039,53.275,494.500,19.095,123.713,0.020,0.070,0.068,...,False,1.17,0.25,1.869878e+07,350800,142809000,57.7,198.3,1.65,6.7


In [61]:
df_Merge = pd.read_pickle("df_Merge.pkl")

df_Merge[["TICKER", "permno",  'month', 'CURR', 'conm', 'ajexm', 'cshtrm',
       'curcdm', 'prccm', 'trfm', 'trt1m', 'cshom', 'ggroup', 'gind',
       'gsector', 'gsubind',"COMNAM",'is_member']].head(10)

df_Merge = df_Merge[df_Merge["ajexm"]!= 0] # LH had 0 ajexm

df_Merge = df_Merge.drop_duplicates(["conm", "month"])
df_Merge.to_csv("df_Merge.csv")
#df_Merge

In [62]:
df_Merge.columns

Index(['TICKER', 'month', 'bm', 'evm', 'pe_exi', 'ps', 'pcf', 'npm', 'roe',
       'aftret_invcapx', 'debt_assets', 'de_ratio', 'at_turn', 'accrual',
       'ptb', 'permno', 'CNAME', 'NUMEST', 'NUMUP4W', 'NUMDOWN4W', 'MEDPTG',
       'STDEV', 'PTGHIGH', 'PTGLOW', 'CURR', 'conm', 'ajexm', 'cshtrm',
       'curcdm', 'prccm', 'trfm', 'trt1m', 'cshom', 'ggroup', 'gind',
       'gsector', 'gsubind', 'COMNAM', 'start', 'ending', 'is_member', 'CPI',
       'FedFundsTargetRate', 'GDP', 'MedianHomeSalesPrice', 'NonFarmPayrolls',
       'PMI', 'PPI', 'PrivateHousingStarts', 'Unemployment'],
      dtype='object')

In [63]:
(df_Merge.count()/len(df_Merge)).sort_values(ascending = False)

TICKER                  1.000000
is_member               1.000000
PTGHIGH                 1.000000
PTGLOW                  1.000000
CURR                    1.000000
month                   1.000000
COMNAM                  1.000000
start                   1.000000
ending                  1.000000
CPI                     1.000000
NUMDOWN4W               1.000000
FedFundsTargetRate      1.000000
GDP                     1.000000
MedianHomeSalesPrice    1.000000
NonFarmPayrolls         1.000000
PMI                     1.000000
PPI                     1.000000
PrivateHousingStarts    1.000000
MEDPTG                  1.000000
conm                    1.000000
NUMUP4W                 1.000000
CNAME                   1.000000
permno                  1.000000
Unemployment            1.000000
NUMEST                  1.000000
ajexm                   0.999884
trfm                    0.999629
curcdm                  0.999526
prccm                   0.999520
trt1m                   0.999003
cshom     

In [64]:
df_Merge[['TICKER','month', 'CURR', 'conm', 'ajexm', 'cshtrm',
       'curcdm', 'prccm', 'trfm', 'trt1m', 'cshom', 'ggroup', 'gind',
       'gsector', 'gsubind',"COMNAM",'is_member']]

,TICKER,month,CURR,conm,ajexm,cshtrm,curcdm,prccm,trfm,trt1m,cshom,ggroup,gind,gsector,gsubind,COMNAM,is_member
0,EGAS,2009-11,USD,ATMOS ENERGY CORP,1.0,1.107772e+07,USD,27.390,4.0436,-0.4488,92600000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,False
1,EGAS,2009-11,USD,GAS NATURAL INC,1.0,1.106310e+05,USD,8.898,3.0673,0.7095,4361000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,False
4,EGAS,2009-12,USD,ATMOS ENERGY CORP,1.0,1.070786e+07,USD,29.400,4.0436,7.3384,92932000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,False
5,EGAS,2009-12,USD,GAS NATURAL INC,1.0,7.103970e+05,USD,10.300,3.0807,16.2621,4361000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,False
8,EGAS,2010-01,USD,ATMOS ENERGY CORP,1.0,6.325268e+06,USD,27.620,4.0436,-6.0544,92932000.0,5510.0,551020.0,55.0,55102010.0,ENERGAS CO,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213397,TSLA,2020-08,USD,TESLA INC,1.0,1.545841e+09,USD,498.320,1.0000,74.1452,931810000.0,2510.0,251020.0,25.0,25102010.0,TESLA MOTORS INC,False
213398,TSLA,2020-09,USD,TESLA INC,1.0,1.705752e+09,USD,429.010,1.0000,-13.9087,933540000.0,2510.0,251020.0,25.0,25102010.0,TESLA MOTORS INC,False
213399,TSLA,2020-10,USD,TESLA INC,1.0,8.267012e+08,USD,388.040,1.0000,-9.5499,947901000.0,2510.0,251020.0,25.0,25102010.0,TESLA MOTORS INC,False
213400,TSLA,2020-11,USD,TESLA INC,1.0,7.702262e+08,USD,567.600,1.0000,46.2736,947901000.0,2510.0,251020.0,25.0,25102010.0,TESLA MOTORS INC,False


# 7) Format data for ML

In [ ]:
df_all = copy(df_Merge)
company_name = "TICKER"
# returns
df_all["mcap"] = df_all["prccm"] * df_all["cshom"]
#use 1m returns rather than prices
#df_all["price_adjusted"] = df_all["prccm"] /df_all["ajexm"]
df_all["return_1M"] = df_all["trt1m"]/100

#melt, then pivot to create price df
#prices = df_all.melt(id_vars = [company_name, "month"], value_vars = "price_adjusted", var_name = "price_adjusted")
#prices = prices.pivot_table(values = "value", index = "month", columns = company_name)
returns = df_all.melt(id_vars = [company_name, "month"], value_vars = "return_1M", var_name = "return_1M")
returns = returns.pivot_table(values = "value", index = "month", columns = company_name)
returns

TICKER,A,AA,AAL,AAP,AAPL,AAS,ABBV,ABC,ABI,ABMD,...,XTO,XYL,YUM,YUMC,Z,ZBH,ZBRA,ZION,ZMH,ZTS
month,,,,,,,,,,,,,,,,,,,,,
2000-01,NaN,NaN,NaN,NaN,0.009119,0.193416,NaN,NaN,NaN,NaN,...,-0.117241,NaN,-0.258900,NaN,-0.142857,NaN,0.011752,0.003844,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,0.104819,-0.196552,NaN,NaN,NaN,NaN,...,0.078125,NaN,-0.069869,NaN,-0.052083,NaN,0.124604,-0.102537,NaN,NaN
2000-03,0.001203,NaN,NaN,NaN,0.184842,0.030043,NaN,NaN,NaN,-0.452240,...,0.515652,NaN,0.166667,NaN,0.571429,NaN,-0.248826,-0.215548,NaN,NaN
2000-04,-0.147837,NaN,NaN,NaN,-0.086516,0.333333,NaN,NaN,NaN,-0.083333,...,0.105263,NaN,0.090543,NaN,0.328671,NaN,0.140000,-0.003003,NaN,NaN
2000-05,-0.169252,NaN,NaN,NaN,-0.322922,0.212500,NaN,NaN,NaN,0.030303,...,0.406926,NaN,-0.134686,NaN,-0.089474,NaN,-0.157895,0.128313,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08,0.042458,0.124615,0.108191,0.041095,0.216309,NaN,0.009061,-0.027348,NaN,0.025605,...,NaN,0.102220,0.057880,0.126269,0.142984,0.044639,0.020588,0.000924,NaN,0.055512
2020-09,0.005178,-0.204514,0.354939,-0.016378,-0.102526,NaN,-0.085413,-0.001134,NaN,-0.099343,...,NaN,0.049139,-0.047470,-0.082481,0.136803,-0.031942,-0.118906,-0.091418,NaN,0.032917
2020-10,0.013176,0.110920,-0.100517,-0.040521,-0.060012,NaN,-0.014956,-0.008770,NaN,-0.090883,...,NaN,0.035901,0.022234,0.005288,-0.003284,-0.029675,0.123505,0.104381,NaN,-0.040031


In [98]:
((1+returns).rolling(window=6, axis = 0).apply(np.prod, raw=True) - 1).head(10)

TICKER,A,AA,AAL,AAP,AAPL,AAS,ABBV,ABC,ABI,ABMD,...,XTO,XYL,YUM,YUMC,Z,ZBH,ZBRA,ZION,ZMH,ZTS
month,,,,,,,,,,,,,,,,,,,,,
2000-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06,NaN,NaN,NaN,NaN,0.018846,1.041153,NaN,NaN,NaN,NaN,...,1.444352,NaN,-0.268608,NaN,0.464286,NaN,-0.242521,-0.217512,NaN,NaN
2000-07,NaN,NaN,NaN,NaN,-0.020482,0.927586,NaN,NaN,NaN,NaN,...,1.057189,NaN,-0.155021,NaN,1.354167,NaN,-0.137276,-0.256869,NaN,NaN
2000-08,-0.419375,NaN,NaN,NaN,0.063250,1.386266,NaN,NaN,NaN,-0.183431,...,2.068949,NaN,0.093898,NaN,1.461537,NaN,-0.188732,-0.149252,NaN,NaN
2000-09,-0.529447,NaN,NaN,NaN,-0.620801,2.133333,NaN,NaN,NaN,0.682100,...,1.205494,NaN,-0.014084,NaN,0.384615,NaN,-0.038750,0.239388,NaN,NaN


In [97]:
((1+returns).rolling(window=6, axis = 0).apply(np.prod, raw=True) - 1).shift(-6).head(10)

TICKER,A,AA,AAL,AAP,AAPL,AAS,ABBV,ABC,ABI,ABMD,...,XTO,XYL,YUM,YUMC,Z,ZBH,ZBRA,ZION,ZMH,ZTS
month,,,,,,,,,,,,,,,,,,,,,
2000-01,NaN,NaN,NaN,NaN,-0.020482,0.927586,NaN,NaN,NaN,NaN,...,1.057189,NaN,-0.155021,NaN,1.354167,NaN,-0.137276,-0.256869,NaN,NaN
2000-02,-0.419375,NaN,NaN,NaN,0.063250,1.386266,NaN,NaN,NaN,-0.183431,...,2.068949,NaN,0.093898,NaN,1.461537,NaN,-0.188732,-0.149252,NaN,NaN
2000-03,-0.529447,NaN,NaN,NaN,-0.620801,2.133333,NaN,NaN,NaN,0.682100,...,1.205494,NaN,-0.014084,NaN,0.384615,NaN,-0.038750,0.239388,NaN,NaN
2000-04,-0.477433,NaN,NaN,NaN,-0.684635,1.171875,NaN,NaN,NaN,0.562290,...,0.956448,NaN,-0.114390,NaN,0.189474,NaN,-0.231360,0.396185,NaN,NaN
2000-05,-0.291172,NaN,NaN,NaN,-0.607143,1.030928,NaN,NaN,NaN,0.218954,...,0.469121,NaN,0.228146,NaN,0.242774,NaN,-0.127605,0.181127,NaN,NaN
2000-06,-0.257627,NaN,NaN,NaN,-0.715991,0.629032,NaN,NaN,NaN,0.577234,...,0.883015,NaN,0.168142,NaN,0.512194,NaN,-0.079338,0.371658,NaN,NaN
2000-07,0.338651,NaN,NaN,NaN,-0.574416,0.369016,NaN,NaN,NaN,0.135218,...,0.890639,NaN,0.489199,NaN,-0.083186,NaN,0.090574,0.287549,NaN,NaN
2000-08,-0.403109,NaN,NaN,NaN,-0.700513,0.545899,NaN,NaN,NaN,-0.246378,...,0.400950,NaN,0.321889,NaN,-0.213572,NaN,-0.165510,0.292706,NaN,NaN
2000-09,-0.372056,NaN,NaN,NaN,-0.142912,0.043617,NaN,NaN,NaN,-0.537615,...,0.290888,NaN,0.247021,NaN,0.115151,NaN,-0.206763,0.025855,NaN,NaN


In [139]:
df_all = copy(df_Merge)
company_name = "TICKER"
# returns
df_all["mcap"] = df_all["prccm"] * df_all["cshom"]
#use 1m returns rather than prices
#df_all["price_adjusted"] = df_all["prccm"] /df_all["ajexm"]
df_all["return_1M"] = df_all["trt1m"]/100

#melt, then pivot to create price df
#prices = df_all.melt(id_vars = [company_name, "month"], value_vars = "price_adjusted", var_name = "price_adjusted")
#prices = prices.pivot_table(values = "value", index = "month", columns = company_name)
returns = df_all.melt(id_vars = [company_name, "month"], value_vars = "return_1M", var_name = "return_1M")
returns = returns.pivot_table(values = "value", index = "month", columns = company_name)

#rets over diff periods
#periods = [-1, 1,3,6,9,12]
#label = ["pred_target","return_1M","return_3M","return_6M","return_9M","return_12M"]
periods = [-1, 3,6,9,12]
label = ["pred_target","return_3M","return_6M","return_9M","return_12M"]
for j in range(len(periods)):
    i = periods[j]
    if i> 0:
        ret = (1+returns).rolling(window=i).apply(np.prod, raw=True) - 1
        df_melt = ret.melt(var_name = company_name, value_name = label[j], ignore_index = False)
        df_all = df_all.merge(df_melt, on = [company_name, "month"],how = "left")
    elif i < 0:
        ret = ((1+returns).rolling(window=-i, axis = 0).apply(np.prod, raw=True) - 1).shift(i)
        df_melt = ret.melt(var_name = company_name, value_name = label[j], ignore_index = False)
        df_all = df_all.merge(df_melt, on = [company_name, "month"],how = "left")
        
df_all

,TICKER,month,bm,evm,pe_exi,ps,pcf,npm,roe,aftret_invcapx,...,PPI,PrivateHousingStarts,Unemployment,mcap,return_1M,pred_target,return_3M,return_6M,return_9M,return_12M
0,EGAS,2009-11,0.850,5.945,11.708,0.498,3.113,0.042,0.103,0.103,...,177.4,0.64,9.9,2.536314e+09,-0.004488,0.118003,NaN,NaN,NaN,NaN
1,EGAS,2009-11,0.850,5.945,11.708,0.498,3.113,0.042,0.103,0.103,...,177.4,0.64,9.9,3.880418e+07,0.007095,0.118003,NaN,NaN,NaN,NaN
2,EGAS,2009-12,0.850,5.945,13.553,0.577,3.604,0.042,0.103,0.103,...,178.1,0.63,9.9,2.732201e+09,0.073384,-0.039738,NaN,NaN,NaN,NaN
3,EGAS,2009-12,0.850,5.945,13.553,0.577,3.604,0.042,0.103,0.103,...,178.1,0.63,9.9,4.491830e+07,0.162621,-0.039738,NaN,NaN,NaN,NaN
4,EGAS,2010-01,0.850,5.945,13.237,0.563,3.520,0.042,0.103,0.103,...,181.9,0.67,9.8,2.566782e+09,-0.060544,0.002840,0.074975,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164486,TSLA,2020-08,0.049,35.254,258.197,18.062,171.660,0.014,0.054,0.061,...,194.3,1.47,8.4,4.643396e+11,0.741452,-0.139087,1.983953,2.729997,6.551682,10.043849
164487,TSLA,2020-09,0.049,35.254,222.285,15.820,150.352,0.014,0.054,0.061,...,195.5,1.54,7.9,4.004980e+11,-0.139087,-0.095499,0.986508,3.093609,4.127657,7.905438
164488,TSLA,2020-10,0.049,35.254,201.057,14.308,135.979,0.014,0.054,0.061,...,196.5,1.61,6.9,3.678235e+11,-0.095499,0.462736,0.356063,1.481457,1.982311,5.160939
164489,TSLA,2020-11,0.039,53.275,494.500,19.095,123.713,0.020,0.070,0.068,...,198.3,1.65,6.7,5.380286e+11,0.462736,0.243252,0.139028,2.398805,3.248570,7.601575


In [148]:
df_nn = copy(df_all)
#Convert indices to 0-2
df_nn["PMI"] = df_nn["PMI"]/100
df_nn["PPI"] = df_nn["PPI"].astype(float)/100

#one hot encode sectors
df_nn["gsector"] = df_nn["gsector"]/5 - 2

#growth measures
df_nn.rename(columns={'cshtrm':'volume'}, inplace=True)
growth_measures = ["GDP", "MedianHomeSalesPrice", "PrivateHousingStarts","NonFarmPayrolls", "volume"]
for ratio in growth_measures:
    if ratio == "volume":  #no need for percentage. Normalise to a fraction
        df_nn[ratio] = df_nn.groupby(company_name)[ratio].apply(lambda x: (x.diff()[1:]/x.shift(1)))
    else:
        df_nn[ratio] = df_nn.groupby(company_name)[ratio].apply(lambda x: (100* x.diff()[1:]/x.shift(1)))

# medptg - adjusted to a return: slightly weird denominator for normalisation
df_nn["MEDPTG"] = np.exp((df_nn["MEDPTG"] - df_nn["prccm"])/df_nn["MEDPTG"])

# convert analyst up/down to percentage
df_nn["PCTUP4W"] = df_nn["NUMUP4W"]/df_nn["NUMEST"]   #use 4 weeks since there are granularity issues with 1M (e.g. NUMDOWN1M > NUMEST for index 180834)
df_nn["PCTDOWN4W"] = df_nn["NUMDOWN4W"]/df_nn["NUMEST"]

#remove first 12 months & last month
df_nn = df_nn[(df_nn["month"]>= "2001-01") & (df_nn["month"]<= "2020-11")].reset_index(drop = True)

#remove index non-members
df_nn = df_nn[df_nn["is_member"] == True]

#remove duplicates
df_nn = df_nn.drop_duplicates(["conm","month"])
df_nn = df_nn.drop_duplicates([company_name,"month"])

#remove unnecessary columns
valid_cols = [company_name, "month", "pred_target","gsector","mcap",
              "debt_assets", "de_ratio", "evm", "pe_exi", 
              "roe", "npm","ps", "ptb", "pcf", "aftret_invcapx",
              "CPI", "PMI", "PPI", "FedFundsTargetRate", "GDP", 
              "MedianHomeSalesPrice", "PrivateHousingStarts","NonFarmPayrolls", "Unemployment",
            "MEDPTG", "PCTUP4W", "PCTDOWN4W", "volume",
             "return_1M", "return_3M", "return_6M", "return_9M", "return_12M"]

df_nn = df_nn[valid_cols]
#remove critical nans
df_nn = df_nn.dropna(axis = 0, subset =["pred_target", "return_1M", "return_3M", "return_6M", "return_9M", "return_12M"])

#fill nans
df_nn.iloc[:,2:]=df_nn.iloc[:,2:].replace(np.nan, df_nn.iloc[:,2:].mean(axis = 0))

#remove 1month ret > 10. That may be noise (e.g. monthly ret of 199!)
df_nn = df_nn[df_nn["pred_target"] < 10]
df_nn = df_nn[df_nn["return_1M"] < 10]


#save
df_nn.to_pickle("df_NeuralNetworkFeatures.pkl")
df_nn.to_csv("df_NeuralNetworkFeatures.csv")
df_nn



,TICKER,month,pred_target,gsector,mcap,debt_assets,de_ratio,evm,pe_exi,roe,...,Unemployment,MEDPTG,PCTUP4W,PCTDOWN4W,volume,return_1M,return_3M,return_6M,return_9M,return_12M
212,SUNW,2007-02,-0.166667,2.0,1.917384e+07,0.551,1.225,25.053,-188.620,-0.069,...,4.5,2.658267,0.769231,0.000000,1.462797,0.043478,6.999992e-08,-5.278261e-07,-0.314286,-0.478261
213,SUNW,2007-03,-0.200000,2.0,1.597820e+07,0.551,1.225,25.053,-184.920,-0.069,...,4.4,2.668177,0.000000,0.000000,0.584903,-0.166667,-2.000005e-01,-1.304353e-01,-0.428572,-0.473685
214,SUNW,2007-04,-0.312500,2.0,1.292800e+07,0.551,1.225,25.053,-160.620,-0.069,...,4.5,2.678123,0.000000,0.000000,-0.298131,-0.200000,-3.043483e-01,-3.333336e-01,-0.428572,-0.597990
215,SUNW,2007-05,0.090909,2.0,8.934090e+06,0.547,1.208,21.879,-408.010,-0.024,...,4.4,2.692627,0.111111,0.444444,0.482997,-0.312500,-5.416669e-01,-5.416668e-01,-0.541667,-0.685715
216,SUNW,2007-06,0.666667,2.0,9.746280e+06,0.547,1.208,21.879,-420.800,-0.024,...,4.6,2.690307,0.000000,0.000000,-0.234375,0.090909,-4.000001e-01,-5.200004e-01,-0.478261,-0.657143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156571,CBOE,2020-07,0.051425,6.0,9.622444e+09,0.351,0.541,14.910,22.430,0.117,...,10.2,1.140747,0.000000,0.500000,-0.163128,-0.059820,-1.145444e-01,-2.836029e-01,-0.231075,-0.187547
156572,CBOE,2020-08,-0.044122,6.0,9.982897e+09,0.360,0.561,14.185,22.065,0.124,...,8.4,1.055180,0.000000,0.500000,0.071894,0.051425,-1.338531e-01,-1.884053e-01,-0.219395,-0.218739
156573,CBOE,2020-09,-0.073513,6.0,9.542427e+09,0.360,0.561,14.185,21.091,0.124,...,7.9,1.079417,0.133333,0.066667,0.091796,-0.044122,-5.508711e-02,-9.080990e-03,-0.260677,-0.225590
156574,CBOE,2020-10,0.128552,6.0,8.782734e+09,0.360,0.561,14.185,19.541,0.124,...,6.9,1.144780,0.062500,0.375000,0.130047,-0.073513,-6.884904e-02,-1.755072e-01,-0.332926,-0.284015


In [149]:
df_nn[df_nn[[company_name,"month"]].duplicated()].sort_values([company_name,"month"])

,TICKER,month,pred_target,gsector,mcap,debt_assets,de_ratio,evm,pe_exi,roe,...,Unemployment,MEDPTG,PCTUP4W,PCTDOWN4W,volume,return_1M,return_3M,return_6M,return_9M,return_12M


In [150]:
df_nn.sort_values(["pred_target"],ascending = False).head(50)

,TICKER,month,pred_target,gsector,mcap,debt_assets,de_ratio,evm,pe_exi,roe,...,Unemployment,MEDPTG,PCTUP4W,PCTDOWN4W,volume,return_1M,return_3M,return_6M,return_9M,return_12M
34523,CNP,2007-03,4.784706,9.0,5.742217e+09,0.912,10.332,8.755000,13.489000,0.112000,...,4.4,0.946207,0.222222,0.000000,0.221571,0.006169,9.234274e-02,2.763403e-01,4.773531e-01,5.673032e-01
9950,APCC,2006-01,2.625000,5.0,1.288600e+05,0.185,0.227,17.531000,26.333000,0.117000,...,4.7,2.716018,0.000000,0.000000,91.306450,0.000000,-6.000000e-01,-2.000001e-01,-5.000000e-01,3.333335e-01
90656,AIG,2009-07,2.449772,6.0,1.768552e+09,0.923,13.978,-3.362000,-0.367000,-1.366000,...,9.5,1.836316,0.000000,0.333333,-0.601833,-0.433621,-5.239129e-01,-4.867186e-01,-6.560205e-01,-9.731130e-01
9946,APCC,2005-09,2.333333,5.0,9.664500e+04,0.185,0.226,15.219000,25.910000,0.130000,...,5.0,2.716772,0.000000,0.000000,-0.970784,-0.400000,-6.250000e-01,2.500000e-07,2.500000e-07,2.500000e-07
58447,APA,2020-03,2.135167,0.0,1.577181e+09,0.723,4.021,20.593000,-0.443000,-0.639000,...,4.4,2.168540,0.035714,0.642857,3.737244,-0.832263,-8.351666e-01,-8.333293e-01,-8.512098e-01,-8.746915e-01
295,JAVA,2009-07,2.000000,2.0,1.819770e+06,0.670,2.032,10.825000,-3.409000,-0.429000,...,9.5,2.715422,0.100000,0.100000,-0.804545,-0.173554,-6.923079e-01,-6.666667e-01,-7.530865e-01,-9.365080e-01
64506,TNL,2009-03,1.780952,3.0,7.455420e+08,0.730,2.845,4.918000,-0.254000,-0.697000,...,8.7,1.068939,0.000000,0.000000,-0.094463,0.138211,-3.518277e-01,-7.274946e-01,-7.604723e-01,-7.921767e-01
96033,MBI,2008-07,1.735245,6.0,1.621155e+09,0.897,8.752,-6.445000,-0.197000,-0.721000,...,5.8,1.406510,0.000000,0.000000,-0.234034,0.350797,-4.298077e-01,-6.174195e-01,-8.597067e-01,-8.917673e-01
9942,APCC,2005-05,1.666667,5.0,9.664500e+04,0.183,0.225,15.628000,27.663000,0.120000,...,5.1,2.716772,0.428571,0.000000,91.306450,-0.625000,1.250000e-07,1.250000e-07,1.250000e-07,-1.250000e-07
9940,APCC,2005-03,1.666667,5.0,9.664500e+04,0.185,0.227,15.549000,29.056000,0.119000,...,5.2,2.716583,0.000000,0.000000,91.306450,0.000000,0.000000e+00,0.000000e+00,-2.500000e-01,-2.500002e-01


In [151]:
df_nn[df_nn["return_1M"] < 10]

,TICKER,month,pred_target,gsector,mcap,debt_assets,de_ratio,evm,pe_exi,roe,...,Unemployment,MEDPTG,PCTUP4W,PCTDOWN4W,volume,return_1M,return_3M,return_6M,return_9M,return_12M
212,SUNW,2007-02,-0.166667,2.0,1.917384e+07,0.551,1.225,25.053,-188.620,-0.069,...,4.5,2.658267,0.769231,0.000000,1.462797,0.043478,6.999992e-08,-5.278261e-07,-0.314286,-0.478261
213,SUNW,2007-03,-0.200000,2.0,1.597820e+07,0.551,1.225,25.053,-184.920,-0.069,...,4.4,2.668177,0.000000,0.000000,0.584903,-0.166667,-2.000005e-01,-1.304353e-01,-0.428572,-0.473685
214,SUNW,2007-04,-0.312500,2.0,1.292800e+07,0.551,1.225,25.053,-160.620,-0.069,...,4.5,2.678123,0.000000,0.000000,-0.298131,-0.200000,-3.043483e-01,-3.333336e-01,-0.428572,-0.597990
215,SUNW,2007-05,0.090909,2.0,8.934090e+06,0.547,1.208,21.879,-408.010,-0.024,...,4.4,2.692627,0.111111,0.444444,0.482997,-0.312500,-5.416669e-01,-5.416668e-01,-0.541667,-0.685715
216,SUNW,2007-06,0.666667,2.0,9.746280e+06,0.547,1.208,21.879,-420.800,-0.024,...,4.6,2.690307,0.000000,0.000000,-0.234375,0.090909,-4.000001e-01,-5.200004e-01,-0.478261,-0.657143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156571,CBOE,2020-07,0.051425,6.0,9.622444e+09,0.351,0.541,14.910,22.430,0.117,...,10.2,1.140747,0.000000,0.500000,-0.163128,-0.059820,-1.145444e-01,-2.836029e-01,-0.231075,-0.187547
156572,CBOE,2020-08,-0.044122,6.0,9.982897e+09,0.360,0.561,14.185,22.065,0.124,...,8.4,1.055180,0.000000,0.500000,0.071894,0.051425,-1.338531e-01,-1.884053e-01,-0.219395,-0.218739
156573,CBOE,2020-09,-0.073513,6.0,9.542427e+09,0.360,0.561,14.185,21.091,0.124,...,7.9,1.079417,0.133333,0.066667,0.091796,-0.044122,-5.508711e-02,-9.080990e-03,-0.260677,-0.225590
156574,CBOE,2020-10,0.128552,6.0,8.782734e+09,0.360,0.561,14.185,19.541,0.124,...,6.9,1.144780,0.062500,0.375000,0.130047,-0.073513,-6.884904e-02,-1.755072e-01,-0.332926,-0.284015


In [135]:
df_nn.isna().sum()

TICKER                     0
month                      0
pred_target                0
gsector                 1990
mcap                      39
debt_assets               73
de_ratio                  73
evm                      382
pe_exi                  1390
roe                     1984
npm                      168
ps                       168
ptb                     1992
pcf                      100
aftret_invcapx          1742
CPI                        0
PMI                        0
PPI                        0
FedFundsTargetRate         0
GDP                        1
MedianHomeSalesPrice       1
PrivateHousingStarts       1
NonFarmPayrolls            1
Unemployment               0
MEDPTG                     9
PCTUP4W                    0
PCTDOWN4W                  0
volume                   536
return_1M                  0
return_3M                  0
return_6M                  0
return_9M                  0
return_12M                 0
dtype: int64

In [134]:
df_nn = df_nn.dropna(axis = 0, subset =["pred_target", "return_1M", "return_3M", "return_6M", "return_9M", "return_12M"])